In [2]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd

%matplotlib inline

In [102]:
df1 = pd.read_csv("../data/shrek1_dialog_w_timestamps.csv", index_col="Unnamed: 0")
df2 = pd.read_csv("../data/shrek2_dialog_w_timestamps.csv", index_col="Unnamed: 0")
df3 = pd.read_csv("../data/shrek3_dialog_w_timestamps.csv", index_col="Unnamed: 0")
dfs = [df1, df2, df3]
grs = []
df = pd.concat(dfs)
for df in [df]:
    

    df['dialog_clean'] = df.dialog.str.replace(r'[,\'".?!~\t]', '')
    df['dialog_clean'] = df.dialog_clean.str.replace('-', ' ')
    df['dialog_clean'] = df.dialog_clean.str.strip()
    df = df[~df.dialog.isnull()]
    replacements = {'CINDERALLA': 'CINDERELLA', 'CAPTAIN HOOKS': 'CAPTAIN HOOK',
                    'PRINCE CHARING': 'PRINCE CHARMING', 'PRINE CHARMING': 'PRINCE CHARMING',
                   'BIG BAD WOLF': 'WOLFIE', 'MOUSE': 'BLIND MOUSE', 'AUDIENCE': 'CROWD',
                   'GINGERBREAD MAN': 'GINGY', 'KING': 'KING HAROLD', 'SHEK': 'SHREK',
                   'QUEEN': 'QUEEN LILLIAN'}
    selected_chars = {'DONKEY', 'SHREK', 'FIONA', 'MUSIC', 'PRINCE CHARMING',
                         'FAIRY GODMOTHER', 'PUSS', 'KING HAROLD', 'ARTIE',
                         'FARQUAAD', 'MERLIN', 'QUEEN LILLIAN', 'GINGY',
                         'SNOW WHITE', 'MIRROR', 'PINOCCHIO', 'GUARD', 'ROBIN HOOD',
                         'RAPUNZEL'}

    df.name = np.where(df.name.isin(replacements.keys()), df.name.map(replacements), df.name)
    df = df[df.name.isin(selected_chars)]

    df['word_count'] = df.dialog_clean.str.split().apply(len)

    word_counts = df.groupby('name').word_count.sum()
    # top_chars = list(df.groupby('name').word_count.sum().drop('SOUND FX').sort_values(ascending=False)[:20].index)
    top_chars = list(df.groupby('name').word_count.sum().sort_values(ascending=False)[:20].index)
    df = df[df.name.isin(top_chars)]

    dfl = df[df.name != df.name.shift(-1)]

    converse = pd.DataFrame({"name1": dfl.name.values, "name2": dfl.name.shift(-1).values})
    converse = converse[(converse.name1!="MUSIC")&(converse.name2!="MUSIC")]

    adj_mat = converse.groupby("name1").name2.value_counts().unstack().fillna(0)

    grs.append(nx.from_pandas_adjacency(adj_mat))

/home/mgray/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [107]:
top_chars

['DONKEY',
 'SHREK',
 'FIONA',
 'MUSIC',
 'PRINCE CHARMING',
 'FAIRY GODMOTHER',
 'PUSS',
 'KING HAROLD',
 'ARTIE',
 'FARQUAAD',
 'MERLIN',
 'QUEEN LILLIAN',
 'GINGY',
 'SNOW WHITE',
 'MIRROR',
 'PINOCCHIO',
 'GUARD',
 'ROBIN HOOD',
 'RAPUNZEL']

In [103]:
[np.mean(list(nx.algorithms.centrality.degree_centrality(i).values())) for i in grs]

[0.50980392156862742]

In [83]:
jsnx = nx.readwrite.json_graph.cytoscape_data(gr)

In [84]:
json_graph = dict()
json_graph.update({'edges': [i['data'] for i in jsnx['elements']['edges']]})
json_graph.update({'nodes': [{'id': i['data']['id'],
                              'word_count': word_counts[i['data']['id']]} 
                             for i in jsnx['elements']['nodes']]})

In [85]:
with open("../data/shrek_3_network.json", "w") as f:
    f.write(str(json_graph))